# Bayesowskie sieci neuronowe

Na dzisiejszych zajęciach opowiem czym są bayesowskie sieci neuronowe, jak wygląda podejście bayesowskie do głębokiego uczenia i jakimi różnicami charakteryzują się modele bayesowskie. 

Klasyczne uczenie modelu możemy interpretować jako przeprowadzanie Maximum Likelihood Estimation, tj. maksymalizujemy prawdopodobieństwo zaobserwowanych danych uwarunkowane parametrami modelu. Jest to podejście frekwentystyczne. 
Analogicznie w podejściu bayesowskim mówimy o Maximum A Posteriori - mamy jakiś prior, czyli jakąś poprzednią wiedzę, oraz obserwacje. Dzięki twierdzeniu Bayesa możemy uaktualnić nasze przekonania nowymi obserwacjami - otrzymujemy wtedy rozkład posterior. 

_Z perspektywy zatwardziałego statystyka używając jakichkolwiek technik regularyzacji przeprowadzamy MAP, a nie MLE, ponieważ dokładamy już jakąś wiedzę którą mamy o rozkładzie._

Inne źródła opisują uczenie bayesowskie inaczej, np. że mamy po prostu modele których wagi są rozkładami prawdopodobieństwa, w przeciwieństwie do ustalonych wartości (point estimate). Istnieją algorytmy typu Bayes-By-Backprop, które faktycznie pozwalają na wytrenowanie sieci w sposób bardzo zbliżony do tego normalnego, ale nie są jedynymi na które warto zwrócić uwagę, lepiej przyjąć perspektywę uczenia się rozkładu w przestrzeni parametrów, aniżeli jednego optymalnego wektora.

Jak to wygląda w praktyce? Co ciekawe, ucząc sieć bayesowską nie musi być wcale mowy o treningu. Tak jak normalnie, musimy najpierw dobrać odpowiednią do zadania architekturę (functional model w [2]), a poza tym również model stochastyczny, czyli priory. Ten krok można uważać za równoważny dobraniu lossa dla normalnej sieci.

Oznaczmy $\theta$ jako wektor parametrów. Jako priory będziemy musieli dobrać $p(\theta)$ oraz $p(y | x, \theta)$ - bazowy rozkład na przestrzeni parametrów i poziom pewności, że dla obserwacji $x$ i modelu $\theta$ wylądujemy w klasie $y$. 


In [ ]:
from copy import deepcopy

from sklearn import datasets
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.convert_parameters import parameters_to_vector, vector_to_parameters

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def iris_draw_plots(X, y, predicted):
    fig = plt.figure(figsize = (16, 5))

    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)

    z1_plot = ax1.scatter(X[:, 0], X[:, 1], c = y)
    z2_plot = ax2.scatter(X[:, 0], X[:, 1], c = predicted)

    plt.colorbar(z1_plot,ax=ax1)
    plt.colorbar(z2_plot,ax=ax2)

    ax1.set_title("REAL")
    ax2.set_title("PREDICTED")

    plt.show()

In [ ]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, shuffle=True, random_state=42)
X_train, X_test = map(lambda t: torch.from_numpy(t).float(), [X_train, X_test])
y_train, y_test = map(lambda t: torch.from_numpy(t).long(), [y_train, y_test])

In [ ]:
regular = nn.Sequential(
    nn.Linear(4, 100),
    nn.ReLU(),
    nn.Linear(100, 3),
    nn.LogSoftmax(dim=0)
)

In [ ]:
optimizer = optim.Adam(regular.parameters(), lr=0.01)

for _step in range(5):
    pre = regular(X_train)
    loss = F.nll_loss(pre, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    pre = regular(X_test)
    _, predicted = torch.max(pre.data, 1)
    total = y_test.size(0)
    correct = (predicted == y_test).sum()
    print('- Accuracy: %f %%' % (100 * float(correct) / total))

In [ ]:

class BayesLinear(nn.Module):
    def __init__(self, prior_mu, prior_sigma, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        self.prior_mu = torch.tensor(prior_mu)
        self.prior_sigma = torch.tensor(prior_sigma)
        self.prior_log_sigma = torch.log(self.prior_sigma)
        
        self.weight_mu = nn.Parameter(torch.empty(out_features, in_features))
        self.weight_log_sigma = nn.Parameter(torch.empty(out_features, in_features))
        self.weight_eps = torch.randn_like(self.weight_log_sigma)
                
        self.bias_mu = nn.Parameter(torch.empty(out_features))
        self.bias_log_sigma = nn.Parameter(torch.empty(out_features))
        self.bias_eps = torch.randn_like(self.bias_log_sigma)
                        
        self.unfreeze()
        self.reset_parameters()

    def reset_parameters(self):
        # Initialization method of Adv-BNN
        stdv = 1. / torch.sqrt(torch.tensor(self.weight_mu.size(1)))
        stdv = stdv.item()
        self.weight_mu.data.uniform_(-stdv, stdv)
        self.weight_log_sigma.data.fill_(self.prior_log_sigma)
        self.bias_mu.data.uniform_(-stdv, stdv)
        self.bias_log_sigma.data.fill_(self.prior_log_sigma)
            
    def forward(self, input):
        if not self._freeze:
            # resample parameters
            self.weight_eps = torch.randn_like(self.weight_log_sigma)
            self.bias_eps = torch.randn_like(self.bias_log_sigma)

        weight = self.weight_mu + torch.exp(self.weight_log_sigma) * self.weight_eps
        bias = self.bias_mu + torch.exp(self.bias_log_sigma) * self.bias_eps

        return F.linear(input, weight, bias)
    
    def freeze(self):
        self._freeze = True
    
    def unfreeze(self):
        self._freeze = False

    def _kl_loss(self, mu_0, log_sigma_0, mu_1, log_sigma_1):
        kl = log_sigma_1 - log_sigma_0 + \
            (torch.exp(log_sigma_0)**2 + (mu_0-mu_1)**2)/(2*torch.exp(log_sigma_1)**2) - 0.5
        return kl.sum()

    def kl_loss(self):
        return self._kl_loss(self.weight_mu, self.weight_log_sigma, self.prior_mu, self.prior_log_sigma) + \
               self._kl_loss(self.bias_mu, self.bias_log_sigma, self.prior_mu, self.prior_log_sigma)

# Zadanie 1
Napisz funkcję, która obliczy średnią dywergencję KL dowolnie głębokiej sieci korzystającej z warstw `BayesLinear`. Chodzi tu o zsumowanie wartości funkcji `kl_loss` dla tych warstw, i podzielenie przez sumaryczną liczbę parametrów (parę (mu, log_sigma) traktujemy jako jeden parametr).

In [ ]:
### Zadanie 1 ###
def kl_loss(model):
    pass

In [ ]:
bayesian = nn.Sequential(
    BayesLinear(0, 0.1, 4, 100),
    nn.ReLU(),
    BayesLinear(0, 0.1, 100, 3),
    nn.LogSoftmax(dim=0)
)

In [ ]:
optimizer = optim.Adam(bayesian.parameters(), lr=0.01)
kl_weight = 0.01

for _step in range(50):
    pre = bayesian(X_train)
    _kl_loss = kl_loss(bayesian)
    nll_loss = F.nll_loss(pre, y_train)
    loss = nll_loss + kl_weight * _kl_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    
    print(
        f'{_kl_loss.item()=:.6f}',
        f'{nll_loss.item()=:.6f}',
        f'{loss.item()=:.6f}',
    sep=' | ')

    pre = bayesian(X_test)
    _, predicted = torch.max(pre.data, 1)
    total = y_test.size(0)
    correct = (predicted == y_test).sum()
    print('- Accuracy: %f %%' % (100 * float(correct) / total))

In [ ]:
regression_model = nn.Sequential(
  BayesLinear(0, 0.1, 1, 100),
  nn.ReLU(),
  BayesLinear(0, 0.1, 100, 1),
)

In [ ]:
X_reg = torch.linspace(-3, 3, steps=500)
y_reg = torch.sin(X_reg)
y_reg += torch.randn_like(y_reg) * 0.1

In [ ]:
def plot_predictions(model):
    xlims = (-3.4, 3.4)
    X_test = torch.linspace(*xlims, 500 + 200)

    predictions = torch.empty((10, *X_test.shape))
    for i in range(10):
        with torch.no_grad():
            predictions[i] = model(X_test.unsqueeze(1)).squeeze()

    std, mean = torch.std_mean(predictions, dim=0)
    fig, ax = plt.subplots(figsize=(12, 5))
    plt.xlim(xlims)
    plt.ylim([-2, 2])
    ax.plot(X_reg, y_reg, 'ko', markersize=4, label="Noisy training data")
    for p in predictions:
        ax.plot(X_test, p, '-', color="purple", linewidth=2, alpha=0.4)
    ax.plot(X_test, mean, '-', color="#408765", linewidth=5, label="Predictions")
    ax.fill_between(X_test, mean - 3 * std, mean + 3 * std, alpha=0.4, color='#86CFAC', zorder=5)

    plt.legend(loc=4, fontsize=15, frameon=False)

In [ ]:
optimizer = optim.Adam(regression_model.parameters(), lr=0.01)
mse = nn.MSELoss()
best_model = regression_model
best_loss = float('inf')

for step in range(1000):
    pre = regression_model(X_reg.unsqueeze(1)).squeeze()
    _kl_loss = kl_loss(regression_model)
    kl_weight = 1 / (step + 1)
    mse_loss = mse(pre, y_train)
    loss = mse_loss + kl_weight * _kl_loss
    
    optimizer.zero_grad()
    loss.backward()
    with torch.no_grad():
        optimizer.step()
        if loss < best_loss:
            best_loss = loss
            best_model = deepcopy(regression_model)


    if step % 250 == 0:
        with torch.no_grad():
            print(
                f'{_kl_loss.item()=:.6f}',
                f'{mse_loss.item()=:.6f}',
                f'{best_loss.item()=:.6f}',
            sep=' | ')

        plot_predictions(best_model)


# Zadanie 2
Jakie wyniki osiągniemy uśredniając nie wyjście, a parametry modelu? Zapisz wnioski i obserwacje.

In [ ]:
best_model.eval()

ensemble = []
predictions = torch.empty((10, *y_train.shape))
for i in range(10):
  with torch.no_grad():
    ensemble.append(deepcopy(best_model.state_dict()))
  

In [ ]:
### Zadanie 2 ###